In [151]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from  torchvision.transforms import transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)


In [152]:
print(torch.__version__)
print(torchvision.__version__)

1.9.1
0.2.2


In [153]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [154]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)      #flatten to get 120
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = t

        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)
        

        t = self.out(t)

        return t


In [155]:
train_set = torchvision.datasets.FashionMNIST(root='/.data/', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))



In [156]:
network = Network()

In [157]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
batch = next(iter(train_loader))
images, labels = batch


# Calculating the loss

In [158]:
preds = network(images)
loss = F.cross_entropy(preds, labels)
loss.item()

2.3044185638427734

# Calculating the gradients

In [159]:
print(network.conv1.weight.grad)

None


In [160]:
loss.backward()

In [161]:
network.conv1.weight.grad.shape

torch.Size([6, 1, 5, 5])

### Updating the weights

In [162]:
optimizer = optim.Adam(network.parameters(), lr=0.01)  # SGD ex. optim.SGD(network....)  # lr=learning rate

In [163]:
loss.item()

2.3044185638427734

In [164]:
get_num_correct(preds, labels)

9

In [165]:
optimizer.step()  #updating the weights

In [166]:
preds = network(images)
loss = F.cross_entropy(preds, labels)

In [167]:
loss.item()

2.265170097351074

In [168]:
get_num_correct(preds, labels)

29

# summarize above code, training 1 batch

In [169]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

batch = next(iter(train_loader))  # Get Batch
images, labels = batch

preds = network(images)  # Pass Batch
loss = F.cross_entropy(preds, labels)  # Calculate Loss

loss.backward()  # Calculate Gradients
optimizer.step()  # Update Weights

print('loss1:', loss.item())
preds = network(images)
loss = F.cross_entropy(preds, labels)
print('loss2:', loss.item())


loss1: 2.302593946456909
loss2: 2.2822041511535645
